In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{explode, col, split}

In [2]:
val spark = SparkSession.builder().appName("Movie Lens Dataset Processing").master("local[8]").getOrCreate()
val sprk_ctx = spark.sparkContext

spark = org.apache.spark.sql.SparkSession@64676e4f
sprk_ctx = org.apache.spark.SparkContext@764b14f8


org.apache.spark.SparkContext@764b14f8

In [3]:
val rating_gcs_path = "gs://artifacts_spark/rating.csv"
val movie_gcs_path = "gs://artifacts_spark/movie.csv"

val rating_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(rating_gcs_path)
val movie_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(movie_gcs_path)

rating_gcs_path = gs://artifacts_spark/rating.csv
movie_gcs_path = gs://artifacts_spark/movie.csv
rating_df = [userId: int, movieId: int ... 2 more fields]
movie_df = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [4]:
rating_df.show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



In [5]:
movie_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [6]:
movie_df.filter(col("genres").isNull).count()

0

#### Use DataFrames to parse and explode the genres column into individual genre rows (e.g., split Action|Comedy into two rows: Action and Comedy).

In [7]:
val movieExplodedGenresDF = movie_df.withColumn("genres", explode(split(col("genres"),"\\|"))).select("movieId","title","genres")

movieExplodedGenresDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [8]:
movieExplodedGenresDF.show(5)

+-------+----------------+---------+
|movieId|           title|   genres|
+-------+----------------+---------+
|      1|Toy Story (1995)|Adventure|
|      1|Toy Story (1995)|Animation|
|      1|Toy Story (1995)| Children|
|      1|Toy Story (1995)|   Comedy|
|      1|Toy Story (1995)|  Fantasy|
+-------+----------------+---------+
only showing top 5 rows



#### Explore the distinct Genres for correction

In [9]:
val uniqueGenres = movieExplodedGenresDF.select("genres").distinct()

uniqueGenres = [genres: string]


[genres: string]

In [10]:
uniqueGenres.show()

+------------------+
|            genres|
+------------------+
|           Mystery|
|           Musical|
|            Action|
|           Romance|
|          Thriller|
|           Fantasy|
|         Animation|
|         Film-Noir|
|              IMAX|
|            Sci-Fi|
|             Drama|
|       Documentary|
|(no genres listed)|
|           Western|
|            Comedy|
|             Crime|
|               War|
|          Children|
|         Adventure|
|            Horror|
+------------------+



#### Convert to an RDD for custom transformations to handle inconsistent genre names (e.g., mapping Sci-Fi to Science Fiction).

In [11]:
val convertedMovieRdd = movieExplodedGenresDF.rdd.map(row => {
  val updatedGenre = row.getString(2) match {
    case "Sci-Fi" => "Science Fiction"
    case "Film-Noir" => "Black & White"
    // case genre if genre.matches("^(no\\s*)$") => "Unknown"  //Regex is not working have to explore on implementing it.  
    case "(no genres listed)" => "Unknown"  
    case other => other  
  }
  (row.getInt(0), row.getString(1), updatedGenre)
})


convertedMovieRdd = MapPartitionsRDD[52] at map at <console>:26


MapPartitionsRDD[52] at map at <console>:26

In [12]:
val processedMovieDf = convertedMovieRdd.toDF().withColumnRenamed("_1", "movieId").withColumnRenamed("_2", "Movie Name").withColumnRenamed("_3", "genres")

processedMovieDf = [movieId: int, Movie Name: string ... 1 more field]


[movieId: int, Movie Name: string ... 1 more field]

#### Perform the join between movies and ratings on movieId with DataFrame.


In [13]:
val joinedDf = processedMovieDf.join(rating_df, "movieId")

joinedDf = [movieId: int, Movie Name: string ... 4 more fields]


[movieId: int, Movie Name: string ... 4 more fields]

In [14]:
joinedDf.show(5)

+-------+--------------------+---------+------+------+-------------------+
|movieId|          Movie Name|   genres|userId|rating|          timestamp|
+-------+--------------------+---------+------+------+-------------------+
|   3918|Hellbound: Hellra...|   Horror|     2|   3.0|2000-11-21 15:35:43|
|   2366|    King Kong (1933)|   Horror|     3|   4.0|1999-12-11 13:18:30|
|   2366|    King Kong (1933)|  Fantasy|     3|   4.0|1999-12-11 13:18:30|
|   2366|    King Kong (1933)|Adventure|     3|   4.0|1999-12-11 13:18:30|
|   2366|    King Kong (1933)|   Action|     3|   4.0|1999-12-11 13:18:30|
+-------+--------------------+---------+------+------+-------------------+
only showing top 5 rows



#### Use RDD transformations to calculate the average rating for each genre using a combination of reduceByKey and custom key-value mapping.

In [15]:
val dataRdd = joinedDf.rdd

dataRdd = MapPartitionsRDD[78] at rdd at <console>:26


MapPartitionsRDD[78] at rdd at <console>:26

In [16]:
val formattedRdd = dataRdd.map(row => {
                            val genre = row.get(2).toString  
                            val rating = row.get(4).toString.toFloat
                            (genre, (1, rating))
                    })

formattedRdd = MapPartitionsRDD[79] at map at <console>:26


MapPartitionsRDD[79] at map at <console>:26

In [17]:
val reducedRdd = formattedRdd.reduceByKey((a_record, b_record) => (a_record._1 + b_record._1, a_record._2+b_record._2))

reducedRdd = ShuffledRDD[80] at reduceByKey at <console>:26


ShuffledRDD[80] at reduceByKey at <console>:26

In [18]:
val finalRdd = reducedRdd.map(record => (record._1, record._2._2 / record._2._1))

finalRdd = MapPartitionsRDD[81] at map at <console>:26


MapPartitionsRDD[81] at map at <console>:26

#### Converting into DF and saving as Parquet

In [19]:
val finalDf = finalRdd.toDF.withColumnRenamed("_1", "Genre").withColumnRenamed("_2", "Average Rating")

finalDf = [Genre: string, Average Rating: float]


[Genre: string, Average Rating: float]

In [20]:
finalDf.coalesce(1).write.mode("overwrite").parquet("hdfs:///user/mano/avgRating.parquet")

In [21]:
finalDf.coalesce(1).show()

+---------------+--------------+
|          Genre|Average Rating|
+---------------+--------------+
|            War|     3.8095307|
|        Musical|     3.5580907|
|Science Fiction|     3.4367728|
|       Thriller|      3.507111|
|      Adventure|     3.5018928|
|         Comedy|      3.426011|
|        Western|      3.570498|
|         Horror|     3.2772238|
|          Crime|     3.6745274|
|        Unknown|      3.006925|
|           IMAX|      3.655946|
|      Animation|     3.6174939|
|    Documentary|     3.7397177|
|        Fantasy|     3.5059454|
|  Black & White|     3.9653811|
|         Action|     3.4438639|
|       Children|     3.4081137|
|        Mystery|      3.663509|
|        Romance|     3.5418024|
|          Drama|     3.6742957|
+---------------+--------------+



In [22]:
spark.stop()